# 전력 수요량 예측 프로젝트 (2025 봄학기 시계열데이터분석 중간보고서)

**✅ 조원 이름: (여기에 조원 이름 작성)**

## 1. 시계열 데이터에 대한 설명

한국전력거래소의 '시간별 전국 전력수요량' 데이터를 사용하였다. 데이터를 wide 포맷에서 long 포맷으로 변환한 후 datetime 인덱스를 추가하여 시계열 데이터로 구성하였다.

## 2. 데이터 탐색 및 분석 포인트

시간대별, 요일별, 월별 전력 사용 패턴을 분석하고, 시계열 분해(Trend, Seasonality, Residuals)를 수행하였다.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# 데이터 불러오기
file_path = "한국전력거래소_시간별 전국 전력수요량_20241231.csv"
df = pd.read_csv(file_path, encoding="cp949")

# wide -> long 변환
df_long = df.melt(id_vars=["날짜"], var_name="시간", value_name="전력수요(MW)")
df_long.loc[df_long["시간"] == "24시", "시간"] = "00시"
df_long.loc[df_long["시간"] == "00시", "날짜"] = pd.to_datetime(df_long.loc[df_long["시간"] == "00시", "날짜"]) + pd.Timedelta(days=1)
df_long["날짜"] = pd.to_datetime(df_long["날짜"])
df_long["datetime"] = pd.to_datetime(df_long["날짜"].dt.strftime('%Y-%m-%d') + " " + df_long["시간"], format="%Y-%m-%d %H시")
df_long = df_long.sort_values("datetime").set_index("datetime")[["전력수요(MW)"]]


In [ ]:
# 시간대, 요일, 월 파생변수 생성
df_long["시간"] = df_long.index.hour
df_long["요일"] = df_long.index.dayofweek
df_long["월"] = df_long.index.month

# 평균 계산
hourly_avg = df_long.groupby("시간")["전력수요(MW)"].mean()
weekday_avg = df_long.groupby("요일")["전력수요(MW)"].mean()
monthly_avg = df_long.groupby("월")["전력수요(MW)"].mean()

# 시각화
fig, axs = plt.subplots(1, 3, figsize=(18, 5))

axs[0].plot(hourly_avg.index, hourly_avg.values, marker='o')
axs[0].set_title("시간대별 평균 전력수요")
axs[0].grid(True)

axs[1].plot(weekday_avg.index, weekday_avg.values, marker='o', color='orange')
axs[1].set_title("요일별 평균 전력수요")
axs[1].grid(True)

axs[2].plot(monthly_avg.index, monthly_avg.values, marker='o', color='green')
axs[2].set_title("월별 평균 전력수요")
axs[2].grid(True)

plt.tight_layout()
plt.show()


## 3. 시계열 데이터 분해 (Trend / Seasonality / Residuals)

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

# 하루(24시간) 주기로 시계열 분해
result = seasonal_decompose(df_long["전력수요(MW)"], model='additive', period=24)
result.plot()
plt.show()


## 4. 시계열 모델 적용

Holt-Winters 가법모델을 사용하여 향후 48시간 전력 수요 예측을 수행하였다.

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

series = df_long["전력수요(MW)"]

# 모델 학습
model = ExponentialSmoothing(series, trend="add", seasonal="add", seasonal_periods=24)
fit = model.fit()

# 48시간 예측
forecast = fit.forecast(48)

# 결과 시각화
plt.figure(figsize=(12,5))
plt.plot(series[-168:], label="최근 7일 실제값")
plt.plot(forecast, label="48시간 예측", linestyle="--")
plt.title("Holt-Winters 모델을 이용한 전력수요 예측")
plt.legend()
plt.grid(True)
plt.show()


## 5. 의미 해석 및 가치 도출

전력 수요량은 시간대, 요일, 월별로 뚜렷한 주기성을 보였으며, Holt-Winters 가법모델을 통해 단기 예측이 가능함을 확인하였다.

향후 기온, 공휴일 등의 외생변수를 추가하면 예측 정확도를 더욱 향상시킬 수 있을 것으로 기대된다.